In [29]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc,Pipeline
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy.linalg import hankel

In [30]:
df_1 = pd.read_csv('~/data/ctown/dataset03.csv')
df_2 = pd.read_csv('~/data/ctown/dataset04.csv')
train_normal=pd.concat((df_1,df_2[df_2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack=df_2[df_2['ATT_FLAG']==1]

In [31]:
hankel = Hankel()
lag = 60
stride = 0.5

In [32]:
sensors=[col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']]


In [33]:
scaler = StandardScaler()
# X_normal=pd.DataFrame(data=scaler.fit_transform(train_normal.loc[:,sensors]),index=train_normal.index,columns=sensors)
X_normal=train_normal.loc[:,sensors]
X_normal

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,0.509730,2.049003,3.191145,2.792634,2.656091,5.316831,1.562321,98.998444,1,99.018150,...,26.426495,87.605774,26.495605,84.206619,18.901676,81.983734,18.791777,67.125603,29.387470,28.487471
1,0.412580,2.009072,3.642565,2.831673,3.126387,5.494855,1.852043,99.095901,1,99.115639,...,26.422962,89.448341,26.487326,85.900085,18.849329,82.150589,18.739643,67.178696,29.354256,28.454256
2,0.320112,1.986093,4.140192,3.256733,3.574601,5.500000,2.246126,98.420959,1,98.440498,...,26.427771,91.056114,26.487364,86.582474,19.597170,83.988579,19.496712,72.425293,29.354538,28.454538
3,0.332879,2.009203,4.673478,3.744497,3.952379,5.500000,3.203573,97.575172,1,97.594460,...,26.519985,92.594353,26.575815,88.020546,26.028486,64.670486,25.922703,76.275040,29.449951,28.549952
4,0.483496,2.089049,5.237937,4.409456,3.504676,5.500000,4.439714,97.351059,1,97.370277,...,26.671642,94.473099,26.723457,90.422462,26.209970,64.746620,26.104692,76.703529,29.574265,28.674263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12441,2.650000,2.370000,3.850000,3.040000,3.820000,4.940000,2.190000,120.080000,1,0.000000,...,27.420000,70.030000,27.380000,84.140000,18.450000,81.670000,18.340000,66.040000,29.880000,28.980000
12442,2.240000,2.560000,3.420000,2.920000,3.690000,5.020000,1.970000,119.120000,1,0.000000,...,27.710000,68.600000,27.660000,83.460000,25.400000,60.850000,25.280000,66.890000,30.190000,29.290000
12443,1.910000,2.760000,2.950000,2.490000,2.700000,5.140000,1.870000,120.710000,1,0.000000,...,26.770000,85.630000,26.840000,82.820000,24.460000,59.560000,24.340000,66.080000,29.680000,28.780000
12444,1.520000,2.520000,3.330000,2.030000,1.690000,5.100000,1.390000,120.020000,1,0.000000,...,25.750000,86.150000,25.780000,103.630000,24.770000,59.010000,24.650000,66.420000,28.980000,28.080000


In [34]:
X_attack=pd.DataFrame(data=scaler.fit_transform(train_attack.loc[:,sensors]),index=train_attack.index,columns=sensors).reset_index().drop(columns=['index'])
X_attack

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,0.510851,-0.546397,-0.878085,-2.084040,-1.526839,-2.239309,-0.533841,-0.623119,0.0,0.548804,...,-2.059407,0.794886,-2.054304,0.013617,-1.323365,0.506761,-1.344766,1.507526,0.763488,-2.086719
1,0.623110,-1.147382,-0.297260,-2.101038,-1.035659,-0.755638,-0.193180,-0.784230,0.0,0.506152,...,-2.298174,0.975545,-2.298500,0.105519,0.590949,1.095478,0.604112,-1.873753,0.980724,-2.343036
2,0.757819,-1.734552,0.341648,-1.710086,-0.312534,0.613905,0.182722,-0.896547,0.0,0.476418,...,-2.578631,0.907219,-2.577037,0.232052,0.696182,1.172992,0.708874,-1.758276,1.095377,-2.634306
3,0.735368,-1.858894,0.995076,-0.911185,0.451524,0.785098,0.770069,-0.477660,0.0,0.587312,...,-0.797353,1.370448,-0.783719,0.457146,0.419171,1.141594,0.433104,-1.461149,-0.877852,-0.618722
4,0.690464,-1.617118,1.503298,-0.078287,1.270157,-0.013802,1.745065,-0.474898,0.0,0.587799,...,-0.069683,-0.380559,-0.089286,0.707547,-1.112899,0.855085,-1.130620,2.164060,-0.657599,-0.051718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,-1.824121,0.323996,-0.268218,-0.316258,1.720405,0.785098,0.558624,-0.129661,0.0,0.679439,...,0.059176,1.170102,0.082415,0.530402,-0.947312,0.784439,-0.944206,0.232086,-0.570101,0.173530
488,-1.741798,0.241102,0.428772,0.431650,0.778977,0.785098,1.145971,-0.233692,0.0,0.651898,...,0.055386,1.181682,0.078599,0.405202,0.106567,-1.176969,0.104953,0.627823,-0.565575,0.185181
489,-1.622056,0.185839,1.111241,0.737612,-0.176095,0.785098,1.944763,-0.284327,0.0,0.638493,...,0.062966,1.538369,0.082415,0.575687,0.573926,-1.154401,0.587165,-1.086172,-0.564067,0.189065
490,-1.427475,0.234194,1.633983,1.434527,-0.899221,-0.812702,0.417661,-0.369946,0.0,0.615827,...,0.532920,-0.321497,0.517390,0.856722,0.654398,-1.162251,0.667277,-1.221112,-0.437345,0.515286


In [35]:
test = pd.read_csv('~/data/ctown/test_dataset.csv')
test

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,04/01/17 00,0.73,2.27,4.00,3.26,3.87,5.50,4.28,98.93,1.0,...,90.54,26.74,84.52,19.43,83.27,19.33,71.33,29.61,28.71,0.0
1,04/01/17 01,0.69,2.25,4.53,3.26,3.84,5.50,4.78,97.95,1.0,...,90.43,26.73,85.04,25.97,64.22,25.86,73.79,29.63,28.73,0.0
2,04/01/17 02,0.90,2.31,5.03,3.41,3.32,5.16,3.22,96.82,1.0,...,91.72,26.89,87.16,29.18,63.81,29.18,59.05,29.80,28.90,0.0
3,04/01/17 03,1.11,2.54,5.16,3.97,2.82,5.01,2.54,96.76,1.0,...,76.12,28.23,88.83,26.53,63.42,26.41,70.92,30.80,29.90,0.0
4,04/01/17 04,1.27,2.99,4.94,4.27,2.35,5.38,3.41,94.77,1.0,...,75.68,31.32,69.55,27.46,63.43,27.34,70.88,33.61,32.71,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,31/03/17 20,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,...,70.07,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0
2085,31/03/17 21,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,...,68.29,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0
2086,31/03/17 22,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,...,87.08,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0
2087,31/03/17 23,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,...,87.85,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0


In [8]:
# X_test_combined=pd.concat((train_attack,test),axis=0).reset_index().drop(columns=['DATETIME','index'])
# X_test_combined

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,3.51,2.55,3.60,2.24,1.62,4.97,2.84,93.19,1.0,93.21,...,86.28,22.50,82.83,16.84,79.83,16.56,80.37,39.76,24.20,1.0
1,3.66,1.68,4.00,2.23,1.98,5.23,3.13,91.44,1.0,91.46,...,87.84,21.86,83.52,29.21,85.83,29.21,54.31,41.20,23.54,1.0
2,3.84,0.83,4.44,2.46,2.51,5.47,3.45,90.22,1.0,90.24,...,87.25,21.13,84.47,29.89,86.62,29.89,55.20,41.96,22.79,1.0
3,3.81,0.65,4.89,2.93,3.07,5.50,3.95,94.77,1.0,94.79,...,91.25,25.83,86.16,28.10,86.30,28.10,57.49,28.88,27.98,1.0
4,3.75,1.00,5.24,3.42,3.67,5.36,4.78,94.80,1.0,94.81,...,76.13,27.65,88.04,18.20,83.38,17.95,85.43,30.34,29.44,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,0.00,...,70.07,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0
2577,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,0.00,...,68.29,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0
2578,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,0.00,...,87.08,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0
2579,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,98.73,...,87.85,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0


In [36]:
# X_test = pd.DataFrame(index=test.index, columns=sensors, data=scaler.fit_transform(test[sensors]))
X_test = test.loc[:,sensors]
Y_test = test.loc[:,'ATT_FLAG']

In [37]:
X_test

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,0.73,2.27,4.00,3.26,3.87,5.50,4.28,98.93,1.0,98.95,...,26.68,90.54,26.74,84.52,19.43,83.27,19.33,71.33,29.61,28.71
1,0.69,2.25,4.53,3.26,3.84,5.50,4.78,97.95,1.0,97.97,...,26.67,90.43,26.73,85.04,25.97,64.22,25.86,73.79,29.63,28.73
2,0.90,2.31,5.03,3.41,3.32,5.16,3.22,96.82,1.0,96.84,...,26.83,91.72,26.89,87.16,29.18,63.81,29.18,59.05,29.80,28.90
3,1.11,2.54,5.16,3.97,2.82,5.01,2.54,96.76,1.0,96.78,...,28.27,76.12,28.23,88.83,26.53,63.42,26.41,70.92,30.80,29.90
4,1.27,2.99,4.94,4.27,2.35,5.38,3.41,94.77,1.0,94.79,...,31.32,75.68,31.32,69.55,27.46,63.43,27.34,70.88,33.61,32.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,0.00,...,27.19,70.07,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76
2085,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,0.00,...,27.12,68.29,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63
2086,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,0.00,...,26.13,87.08,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08
2087,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,98.73,...,26.53,87.85,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59


In [38]:
Y_test

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2084    0.0
2085    0.0
2086    0.0
2087    0.0
2088    0.0
Name: ATT_FLAG, Length: 2089, dtype: float64

In [39]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [40]:
y_actual

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0])

In [41]:
# X_test_1 = pd.DataFrame(index=X_test_combined.index, columns=sensors, data=scaler.fit_transform(X_test_combined[sensors]))
# Y_test_1 = X_test_combined.loc[:,'ATT_FLAG']

In [42]:
# labels_1 = hankel.fit(np.array(Y_test_1),lag,stride)
# y_actual_1 = np.any(labels_1>0,axis=0).astype(int)

In [43]:
# y_actual_1

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [44]:
sensor_models_1 = []
sensor_predicted_1 = []
accuracy_1 = []
precision_1 = []
recall_1 = []
fscore_1 = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted_1.append(y_predicted)
    accuracy_1.append(accuracy_score(y_actual,y_predicted))
    precision_1.append(precision_score(y_actual,y_predicted))
    recall_1.append(recall_score(y_actual,y_predicted))
    fscore_1.append(f1_score(y_actual,y_predicted))
    sensor_models_1.append(model)    

In [45]:
sensor_predicitions_1 = np.asarray(sensor_predicted_1)


In [46]:
sensor_predicitions_1

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [47]:
predicted_label=np.any(sensor_predicitions_1>0,axis=0).astype(int)

In [48]:
predicted_label

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0])

In [49]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.7058823529411765
Precision  0.6216216216216216
Recall  0.7931034482758621
F1-score  0.696969696969697


In [20]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy_1).mean(), np.median(np.asarray(accuracy_1)),np.asarray(accuracy_1).min(), np.asarray(accuracy_1).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision_1).mean(), np.median(np.asarray(precision_1)),np.asarray(precision_1).min(), np.asarray(precision_1).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall_1).mean(), np.median(np.asarray(recall_1)),np.asarray(recall_1).min(), np.asarray(recall_1).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore_1).mean(), np.median(np.asarray(fscore_1)),np.asarray(fscore_1).min(), np.asarray(fscore_1).max()))

Accuracy -  Mean: 0.5310533515731876 Median : 0.5411764705882353 Min : 0.49411764705882355 Max : 0.611764705882353
precision - Mean: 0.6564031643591007 Median : 1.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.08869659275283938 Median : 0.09302325581395349 Min : 0.0 Max : 0.3023255813953488
f1 -        Mean: 0.15072680635847457 Median : 0.1702127659574468 Min : 0.0 Max : 0.41935483870967744


# Multiple clusters: threshold tuning + no concat of training and test data

In [21]:
sensor_models_2= []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,tune=True)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models_2.append(model)    

In [22]:
sensor_predicitions=np.asarray(sensor_predicted)

In [23]:
sensor_predicitions

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 1],
       [1, 0, 1, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [24]:
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)

In [25]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969


In [26]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.570109439124487 Median : 0.5735294117647058 Min : 0.4852941176470588 Max : 0.6764705882352942
precision - Mean: 0.34810005945961203 Median : 0.42857142857142855 Min : 0.0 Max : 0.8
recall -    Mean: 0.18765036086607859 Median : 0.10344827586206896 Min : 0.0 Max : 0.5517241379310345
f1 -        Mean: 0.2241284094771061 Median : 0.16666666666666663 Min : 0.0 Max : 0.5416666666666666
